In [2]:
import numpy as np

def is_pos_def(x):
    counter = 0
    for i in np.linalg.eigvals(x):
        #print(i)
        if i < -0.000000001:
            counter += 1
    if counter == 0:
        return True
    else:
        return False

In [3]:
import pandas as pd
from dtaidistance import dtw

dataset = pd.read_csv('wine.txt', sep = '  ', engine='python', header = None)
dataset = dataset.iloc[:, :-1].values
data_train = dataset[:, 1:101]
target_train = dataset[:, 0]

In [4]:
cou = 0
cov = 0
for i in target_train:
    if i == 1:
        cou += 1
    elif i ==
        cov += 1

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [146]:
import pandas as pd
from dtaidistance import dtw

dataset = pd.read_csv('wine.txt', sep = '  ', engine='python', header = None)
dataset = dataset.iloc[:, :-1].values
data_train = dataset[:, 1:101]
target_train = dataset[:, 0]
l = len(data_train)
D_train = np.zeros((l, l))
for i in range(l):
    for j in range(l):
        D_train[i][j] = dtw.distance(data_train[i], data_train[j])
print(D_train)


[[0.         0.07605102 0.1179844  ... 0.23702952 0.25069143 0.2368523 ]
 [0.07605102 0.         0.0524061  ... 0.22504921 0.23135172 0.22448142]
 [0.1179844  0.0524061  0.         ... 0.21337447 0.2130119  0.21202211]
 ...
 [0.23702952 0.22504921 0.21337447 ... 0.         0.04356678 0.03084929]
 [0.25069143 0.23135172 0.2130119  ... 0.04356678 0.         0.02951089]
 [0.2368523  0.22448142 0.21202211 ... 0.03084929 0.02951089 0.        ]]


In [147]:
is_pos_def(D_train)

False

In [148]:
import math
A_train = np.zeros((l, l))
for i in range(l):
    for j in range(l):
        A_train[i][j] = math.exp(-D_train[i][j])

print(A_train)

[[1.         0.92676893 0.88870992 ... 0.788968   0.77826248 0.78910783]
 [0.92676893 1.         0.94894342 ... 0.79847692 0.79346034 0.79893042]
 [0.88870992 0.94894342 1.         ... 0.80785356 0.80814652 0.80894681]
 ...
 [0.788968   0.79847692 0.80785356 ... 1.         0.95736862 0.96962169]
 [0.77826248 0.79346034 0.80814652 ... 0.95736862 1.         0.9709203 ]
 [0.78910783 0.79893042 0.80894681 ... 0.96962169 0.9709203  1.        ]]


In [149]:
is_pos_def(A_train)

True

In [150]:
import numpy as np
import matplotlib.pyplot as plt

from functools import partial
from sklearn import svm
from sklearn.model_selection import train_test_split

# 200 x 2d timeseries of length 100
#X = np.random.randn(200, 100, 2)
#y = np.random.randint(0, 2, 200)


X_train, X_test, y_train, y_test = train_test_split(data_train, target_train, train_size=0.8)

x_shape = X_train.shape


In [153]:
def kernel_matrix(X1, X2, f):
    D = [[f(X1[i], X2[j]) for j in range(len(X2))]for i in range(len(X1))]
    return np.array(D)

#kernel_matrix(X_test, X_test, lambda x, y: ((x - y)**2).sum())

In [154]:
clf = svm.SVC(kernel="precomputed")
clf.fit(kernel_matrix(X_train, X_train, lambda x, y: ((x - y)**2).sum()), y_train)

print(f"Train score with l2: {clf.score(kernel_matrix(X_train, X_train, lambda x, y: ((x - y)**2).sum()), y_train):.3f}")
print(f"Test score with l2: {clf.score(kernel_matrix(X_test, X_train, lambda x, y: ((x - y)**2).sum()), y_test):.3f}")

Train score with l2: 0.500
Test score with l2: 0.565


In [159]:
clf = svm.SVC(kernel="precomputed")
D = [[dtw.distance(X_train[i], X_train[j]) for j in range (len(X_train))]for i in range(len(X_train))]
C = [[dtw.distance(X_test[i], X_train[j]) for j in range (len(X_train))]for i in range(len(X_test))]
clf.fit(np.array(D), y_train)

print(f"Train score with DTW: {clf.score(np.array(D), y_train):.3f}")
print(f"Test score with DTW: {clf.score(np.array(C), y_test):.3f}")

Train score with DTW: 0.477
Test score with DTW: 0.565


In [161]:
A = np.zeros((len(X_train), len(X_train)))
for i in range(len(X_train)):
    for j in range(len(X_train)):
        A[i][j] = math.exp(-D[i][j])

clf = svm.SVC(kernel="precomputed")

clf.fit(np.array(A), y_train)

print(f"Train score with DTW (upgrade): {clf.score(np.array(A), y_train):.3f}")
print(f"Test score with DTW: {clf.score(np.array(C), y_test):.3f}")


Train score with DTW (upgrade): 0.761
Test score with DTW: 0.304


In [1]:
print(y_train, y_test)

NameError: name 'y_train' is not defined